In [4]:
import sys
!{sys.executable} -m pip install quantum-image-classifier==0.2.11

  Using cached quantum_image_classifier-0.2.11-py3-none-any.whl


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
qiskit 0.30.0 requires qiskit-terra==0.18.2, but you have qiskit-terra 0.20.2 which is incompatible.
qiskit-aqua 0.9.5 requires h5py<3.3.0, but you have h5py 3.6.0 which is incompatible.


  Using cached sklearn-0.0-py2.py3-none-any.whl
  Using cached qiskit_terra-0.20.2-cp39-cp39-win_amd64.whl (6.1 MB)
  Attempting uninstall: qiskit-terra
    Found existing installation: qiskit-terra 0.18.2
    Uninstalling qiskit-terra-0.18.2:
      Successfully uninstalled qiskit-terra-0.18.2


# Quantum image classifier
## Package usage

If you want to use this package you can install it using `pip install quantum-image-classifier`, just as we did on the top of this notebook. After that, you can easily import it as any other package.

In [5]:
import quantum_image_classifier

## Features
In this package you can do two things: treat data to feed into the algorithm and call a quantum algorithm to classify the data. We see how in the next sections.

### Data treatment
You can obtain the data you want to test the algorithms in this section of the package. First of all, you can generate synthetic data by using the function `generate_synthetic_data` like we show you here.

In [6]:
from quantum_image_classifier import data_generator as dg

n_dim = 8
n_clusters = 4
n_samples = 250

train_X_syn, train_y_syn, test_X_syn, test_y_syn = dg.generate_synthetic_data(n_dim, n_clusters, n_samples)

Other option is to use the MNIST dataset. To do this you need to be aware of a very important aspect of the nature of the quantum algorithms: **you can operate with a very limited number of qubits**. This means that, if you wish to classify an actual image, you need to preprocess it before applying the algorithms in order to reduce its dimensionality and be able to use the data (we recommend this to data with a dimension greater than 32). To do this to the MNIST dataset you can use one of the three options we give you:

1. **Use PCA (PCA).** You can use PCA method to achieve a dimension reduction. The only problem this method has is that you have to assume a linear relationship between the input and the output, so, if there is a non-linear relationship, you can generate non-accurate data.
2. **Use simple autoencoder (AE).** You can use a simple autoencoder in order to achieve a dimension reduction. This solves the non-linear cases, but increases the execution time, because you have to train a neural network.
3. **Use autoencoder based on CNN (AE_CNN).** With this less simple implementation of the autoencoder you will achieve much more accuracy than with the simple autoencoder by trading time of execution and complexity.

You have an example of how can you apply this preprocess to the data below, using the function `get_MNIST`.

In [7]:
from quantum_image_classifier import data_loader as dl

# It will take time to compute this operations.

train_X_pca, train_y_pca, test_X_pca, test_y_pca = dl.get_MNIST(8, "PCA")
print("PCA PREPROCESS")
print(train_X_pca[:10])

train_X_ae, train_y_ae, test_X_ae, test_y_ae = dl.get_MNIST(8, "AE")
print("SIMPLE AUTOENCODER PREPROCESS")
print(train_X_ae[:10])

train_X_ae_cnn, train_y_ae_cnn, test_X_ae_cnn, test_y_ae_cnn = dl.get_MNIST(8, "AE_CNN")
print("CNN AUTOENCODER PREPROCESS")
print(train_X_ae_cnn[:10])

PCA PREPROCESS
[[17.14560578 13.25298012 18.13535823 10.01665586 17.08076227 19.02707655
  18.39508841 19.43307664]
 [26.77675236 10.31341219 14.61978148 16.39940107 18.9017336  12.92803099
  17.80613216 21.20379746]
 [20.39617369 27.49921412 11.88374558 19.79191593 22.16280173 20.34857055
  36.80061212 22.12381581]
 [11.48560397 14.32145773 21.75857645 17.60687219 23.69492199 21.53003712
  19.63432352 13.46809908]
 [12.8845245  21.20114262 11.78978295 19.52686599 16.45984685 17.45389807
  17.50771925 14.87889984]
 [20.26617523 14.99937989 17.83656582 20.82033252 16.85115002 23.86059128
  17.37528579 18.41785907]
 [11.29442768 18.8909144  21.00013684 19.05579518 19.52942597 13.11466408
  17.93044163 19.04703404]
 [19.99468052 12.66150551 17.90475725  6.91493292 13.06366412 23.17525268
  18.62990847 16.14010938]
 [10.2454388  18.2596357  19.16789805 19.62687344 22.28994826 12.96897887
  18.70157069 18.49551642]
 [14.84013951 13.34137056 13.77889234 21.69858477 17.3573239  21.59063733
  

### Algorithms
#### Nearest centroid algorithm
In order to use this algorithm, first we need to instance the class NearestCentroid. We can instance this class in two ways: either by giving the training set as an argument (along with the dimension and the labels associated to this dataset) or empty. In the first way, we will need to send the dataset with the associated labels to the function `fit`.

In [ ]:
from quantum_image_classifier import NearestCentroid

# Empty instance
nearest_centroid = NearestCentroid()
nearest_centroid.fit(train_X_syn, train_y_syn, n_dim)
labels_predicted = nearest_centroid.predict(test_X_syn)

print(test_y_syn)
print(labels_predicted)

On the contrary, if we have already given the dataset with its associated labels on the construction, we just need to call fit wthout arguments. By the way, if you call here `fit` with arguments, then you will override the data of the construction, naturally.

In [ ]:
# Instance with dataset and labels.
nearest_centroid = NearestCentroid(train_X_syn, train_y_syn, n_dim)
nearest_centroid.fit()
labels_predicted = nearest_centroid.predict(test_X_syn)

print(test_y_syn)
print(labels_predicted)